# Figure 6

In [ ]:
options(warn=-1)

In [ ]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
        
            # Seurat 
            library(Seurat), 

            # TradeSeq
            library(tradeSeq),

            # CCI 
            library(CellChat), 

            # Data 
            library(tidyverse), 
            library(data.table), 

            # miloR
            library(miloR), 
            library(ggbeeswarm), 

            # Plotting 
            library(ggplot2), 
            library(patchwork), 
            library(cowplot), 
            library(ComplexHeatmap), 

            # Pyhton compatibility
            library(reticulate)

        )
    )
)

In [ ]:
# Configure reticulate 
# use_condaenv(condaenv="p.3.10.16-FD20200109SPLENO", conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)
# py_config()

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source
source("bin/so_pl.R")
source("bin/dea_pp.R")
source("bin/dea_pl.R")
source("bin/gsea_pp.R")
source("bin/gsea_pl.R")
source("bin/cci_pp.R")
source("bin/cci_pl.R")

In [ ]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=4)) # From project global source()

# Import scRNAseq data 

In [ ]:
so <- readRDS("data/scRNAseq/object/pp_1.rds")
so <- NormalizeData(so)

# Differential expression analysis (DEA)

In [ ]:
# Response gene DEA 
contrasts_vec <- c("IFNAR_fl_CpG_D1", "IFNAR_fl_Baseline_D0")
dea_res_1 <- readRDS(paste0("result/dea/scRNAseq/wilcox/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

contrasts_vec <- c("IFNAR_fl_LysM_cre_CpG_D1", "IFNAR_fl_LysM_cre_Baseline_D0")
dea_res_2 <- readRDS(paste0("result/dea/scRNAseq/wilcox/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

contrasts_vec <- c("IFNAR_fl_CpG_D3", "IFNAR_fl_Baseline_D0")
dea_res_3 <- readRDS(paste0("result/dea/scRNAseq/wilcox/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

contrasts_vec <- c("IFNAR_fl_LysM_cre_CpG_D3", "IFNAR_fl_LysM_cre_Baseline_D0")
dea_res_4 <- readRDS(paste0("result/dea/scRNAseq/wilcox/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

# Genotype DEA
contrasts_vec <- c("IFNAR_fl_LysM_cre_Baseline_D0", "IFNAR_fl_Baseline_D0")
dea_res_5 <- readRDS(paste0("result/dea/scRNAseq/wilcox/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

contrasts_vec <- c("IFNAR_fl_LysM_cre_CpG_D1", "IFNAR_fl_CpG_D1")
dea_res_6 <- readRDS(paste0("result/dea/scRNAseq/wilcox/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

contrasts_vec <- c("IFNAR_fl_LysM_cre_CpG_D3", "IFNAR_fl_CpG_D3")
dea_res_7 <- readRDS(paste0("result/dea/scRNAseq/wilcox/sample_group/", contrasts_vec[1], "_vs_", contrasts_vec[2], ".rds"))

In [ ]:
adj_pval_thr <- 0.05
log2_thr <- 0.50

# DEG count overview

## Response count D1 and D3 vs Ctl 

In [ ]:
mat_1 <- lapply(dea_res_1, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t() 
mat_2 <- lapply(dea_res_2, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t() 
mat_3 <- lapply(dea_res_3, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t() 
mat_4 <- lapply(dea_res_4, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t() 

In [ ]:
mat <- list(mat_1, mat_2, mat_3, mat_4)

In [ ]:
mat <- lapply(mat, function(x) {
    
    x <- as.data.frame(x)
    set_diff <- setdiff(make.names(levels(so$celltype_low)), rownames(x))
    x[set_diff, ] <- 0
    x[make.names(levels(so$celltype_low)), , drop=FALSE]

}
             )

mat <- do.call(cbind, mat)

In [ ]:
colnames(mat) <- c("+/+", "cre/+", "+/+", "cre/+")
col_split <- c("D1", "D1", "D3", "D3")

In [ ]:
pdf("result/figures/figure_6/hm_deg_count_d1_d3_vs_ctl_cg_and_ko.pdf", width=1.5, height=3)
source("bin/dea_pl.R")
draw(dea_res_count_hm(as.matrix(mat), col_split=col_split, fontsize_select=2))

dev.off()

## Genotype comparison 

In [ ]:
mat_1 <- lapply(dea_res_5, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t() 
mat_2 <- lapply(dea_res_6, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t() 
mat_3 <- lapply(dea_res_7, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2_thr) %>% dplyr::pull(gene) %>% length}) %>% data.frame() %>% t() 

In [ ]:
mat <- list(mat_1, mat_2, mat_3)

In [ ]:
mat <- lapply(mat, function(x) {
    
    x <- as.data.frame(x)
    set_diff <- setdiff(make.names(levels(so$celltype_low)), rownames(x))
    x[set_diff, ] <- 0
    x[make.names(levels(so$celltype_low)), , drop=FALSE]

}
             )

mat <- do.call(cbind, mat)

In [ ]:
colnames(mat) <- c("+/+ vs cre/+", "+/+ vs cre/+", "+/+ vs cre/+")
col_split <- c("Ctl", "D1", "D3")

In [ ]:
pdf("result/figures/figure_6/hm_deg_count_cg_vs_ko.pdf", width=1.5, height=3)

draw(dea_res_count_hm(as.matrix(mat), col_split=col_split, fontsize_select=2))

dev.off()

# Gene set enrichment analysis (GSEA)

## genotype comparison - M7

In [ ]:
compute_gsea <- FALSE
celltype_select <- c("MEP", "Proerythroblast", "Erythroblast", "cMo", "intMo", "ncMo", "RPM", "cDC1", "cDC2", "moDC", "cDC.mig.", "Basophil")

In [ ]:
if(compute_gsea) {

    # Collapse gene set
    gene_set <- msigdbr::msigdbr(species="mouse", db_species="MM", category="M7", subcategory=NULL)
    gene_set <- gene_set[grep("_UP", gene_set$gs_name), ]
    gene_set$gs_name <- sapply(strsplit(gene_set$gs_name, "_"), function(x) {paste(x[max(1, length(x)-3):(length(x)-2)], collapse="_")})
    gene_set$gs_name <- gsub("CELL_|CD4_|CD8_|MONOCYTE_|MACROPHAGE_|TREG_|LANGERHANS_|NEUTROPHIL_|CDC1_|CDC2_|MIGDC_|ETAC_|GD_|ILC_|PDC_", "", gene_set$gs_name)
    gene_set <- gene_set %>% dplyr::select(gene_symbol, gs_name) %>% dplyr::distinct()

    gsea_res <- lapply(celltype_select, function(i) {
    
        gsea_res_1 <- gsea(dea_res_5[[i]], gene_set=gene_set) %>% dplyr::mutate(col_split="Ctl", col_label=i)
        gsea_res_2 <- gsea(dea_res_6[[i]], gene_set=gene_set) %>% dplyr::mutate(col_split="D1", col_label=i)
        gsea_res_3 <- gsea(dea_res_7[[i]], gene_set=gene_set) %>% dplyr::mutate(col_split="D3", col_label=i)
    
        gsea_res <- rbind(gsea_res_1, gsea_res_2, gsea_res_3)
    
        return(gsea_res)
        
    }
          )

    saveRDS(gsea_res, "result/gsea/scRNAseq/gsea_ctl_d1_d3_wt_vs_ko_m7.rds")
    
} else {

    gsea_res <- readRDS("result/gsea/scRNAseq/gsea_ctl_d1_d3_wt_vs_ko_m7.rds")
    
}

In [ ]:
gsea_res <- do.call(rbind, gsea_res)

In [ ]:
gs_name_select <- gsea_res %>% dplyr::filter(padj<=0.05 & abs(NES)>=1) %>% dplyr::pull(pathway)

In [ ]:
mat_1 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=padj,
        values_fill=1
    
    )

In [ ]:
mat_2 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=NES,
        values_fill=0
    
    )

In [ ]:
row_split <- read.csv("data/reference/immune_dictionary/immune_dictionary.csv") %>% tibble::column_to_rownames("gs_name")
col_label <- mat_1$col_label
col_split <- mat_1$col_split

In [ ]:
mat_1 <- mat_1 %>% dplyr::select(-col_label, -col_split) %>% t()
mat_2 <- mat_2 %>% dplyr::select(-col_label, -col_split) %>% t()

In [ ]:
row_split <- row_split[rownames(mat_1), ]

In [ ]:
rownames(mat_1) <- row_split$norm
rownames(mat_2) <- row_split$norm

In [ ]:
row_split <- row_split %>% dplyr::select(norm, family) %>% dplyr::rename(row_split_label=family) %>% dplyr::distinct()
rownames(row_split) <- row_split$norm

In [ ]:
col_label[col_label=="cDC.mig."] <- "cDC mig."

In [ ]:
pdf("result/figures/figure_6/hm_gsea_ctl_d1_d3_wt_vs_ko_m7.pdf", width=8, height=12)

gsea_hm(mat_1, mat_2, col_label=col_label, col_split=col_split, row_split=row_split, fontsize_select=2)

dev.off()

## genotype comparison - HM

In [ ]:
compute_gsea <- FALSE
celltype_select <- c("MEP", "Proerythroblast", "Erythroblast", "cMo", "intMo", "ncMo", "RPM", "cDC1", "cDC2", "moDC", "cDC.mig.", "Basophil")

In [ ]:
if(compute_gsea) {

    gsea_res <- lapply(celltype_select, function(i) {
    
        gsea_res_1 <- gsea(dea_res_5[[i]], category="MH", subcategory=NULL, gene_set=NULL) %>% dplyr::mutate(col_split="Ctl", col_label=i)
        gsea_res_2 <- gsea(dea_res_6[[i]], category="MH", subcategory=NULL, gene_set=NULL) %>% dplyr::mutate(col_split="D1", col_label=i)
        gsea_res_3 <- gsea(dea_res_7[[i]], category="MH", subcategory=NULL, gene_set=NULL) %>% dplyr::mutate(col_split="D3", col_label=i)
    
        gsea_res <- rbind(gsea_res_1, gsea_res_2, gsea_res_3)
    
        return(gsea_res)
        
    }
          )

    saveRDS(gsea_res, "result/gsea/scRNAseq/gsea_ctl_d1_d3_wt_vs_ko_mh.rds")
    
} else {

    gsea_res <- readRDS("result/gsea/scRNAseq/gsea_ctl_d1_d3_wt_vs_ko_mh.rds")
    
}

In [ ]:
gsea_res <- do.call(rbind, gsea_res)

In [ ]:
gs_name_select <- gsea_res %>% dplyr::filter(padj<=0.05 & abs(NES)>=1) %>% dplyr::pull(pathway)

In [ ]:
mat_1 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=padj,
        values_fill=1
    
    )

In [ ]:
mat_2 <- gsea_res %>% dplyr::filter(pathway %in% gs_name_select) %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=NES,
        values_fill=0
    
    )

In [ ]:
col_label <- mat_1$col_label
col_split <- mat_1$col_split

In [ ]:
mat_1 <- mat_1 %>% dplyr::select(-col_label, -col_split) %>% t()
mat_2 <- mat_2 %>% dplyr::select(-col_label, -col_split) %>% t()

In [ ]:
col_label[col_label=="cDC.mig."] <- "cDC mig."

In [ ]:
pdf("result/figures/figure_6/hm_gsea_ctl_d1_d3_wt_vs_ko_mh.pdf", width=8, height=10)

gsea_hm(mat_1, mat_2, col_label=col_label, col_split=col_split, row_split=NULL, pathway_suffix="HALLMARK", fontsize_select=2)

dev.off()

## genotype comparison - GO

In [ ]:
compute_gsea <- FALSE
celltype_select <- c("MEP", "Proerythroblast", "Erythroblast", "cMo", "intMo", "ncMo", "RPM", "cDC1", "cDC2", "moDC", "cDC.mig.", "Basophil")

In [ ]:
if(compute_gsea) {

    gsea_res <- lapply(celltype_select, function(i) {
    
        gsea_res_1 <- gsea_cp(dea_res_5[[i]]) %>% dplyr::mutate(col_split="Ctl", col_label=i)
        gsea_res_2 <- gsea_cp(dea_res_6[[i]]) %>% dplyr::mutate(col_split="D1", col_label=i)
        gsea_res_3 <- gsea_cp(dea_res_7[[i]]) %>% dplyr::mutate(col_split="D3", col_label=i)
    
        gsea_res <- rbind(gsea_res_1, gsea_res_2, gsea_res_3)
    
        return(gsea_res)
        
    }
          )

    saveRDS(gsea_res, "result/gsea/scRNAseq/gsea_ctl_d1_d3_wt_vs_ko_gobp.rds")
    
} else {

    gsea_res <- readRDS("result/gsea/scRNAseq/gsea_ctl_d1_d3_wt_vs_ko_gobp.rds")
    
}

In [ ]:
gsea_res <- do.call(rbind, gsea_res)

In [ ]:
gs_name_select <- gsea_res %>% dplyr::filter(p.adjust<=0.05 & abs(NES)>=1) %>% dplyr::pull(ID)
gsea_res <- gsea_res %>% dplyr::filter(ID %in% gs_name_select)

In [ ]:
gsea_res <- gsea_res[!gsea_res$Description %in% grep("T cell|B cell|CD4|CD8|lymphocyte|endothelial|epithelial", gsea_res$Description, value=TRUE), ]

In [ ]:
if(compute_gsea) {

    gs_reduce <- gsea_cp_reduce(gsea_res)

    saveRDS(gs_reduce, "result/gsea/scRNAseq/gsea_reduce_ctl_d1_d3_wt_vs_ko_gobp.rds")
    
} else {

    gs_reduce <- readRDS("result/gsea/scRNAseq/gsea_reduce_ctl_d1_d3_wt_vs_ko_gobp.rds")
    
}

In [ ]:
gsea_res <- gsea_res %>% dplyr::rename(pathway=Description, padj=p.adjust) %>% dplyr::filter(ID %in% gs_reduce$go)

In [ ]:
gs_name_select <- gsea_res %>% dplyr::filter(padj<=0.05 & abs(NES)>=2) %>% dplyr::pull(ID)
gsea_res <- gsea_res %>% dplyr::filter(ID %in% gs_name_select)

In [ ]:
mat_1 <- gsea_res %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=padj,
        values_fill=1
    
    )

In [ ]:
mat_2 <- gsea_res %>% 

    tidyr::pivot_wider(
        
        id_cols=c(col_label, col_split),
        names_from=pathway,
        values_from=NES,
        values_fill=0
    
    )

In [ ]:
col_label <- mat_1$col_label
col_split <- mat_1$col_split

In [ ]:
mat_1 <- mat_1 %>% dplyr::select(-col_label, -col_split) %>% t()
mat_2 <- mat_2 %>% dplyr::select(-col_label, -col_split) %>% t()

In [ ]:
row_split <- gs_reduce %>% dplyr::select(term, parentTerm) %>% dplyr::rename(row_split_label=parentTerm) %>% dplyr::distinct()
rownames(row_split) <- gs_reduce$term

In [ ]:
col_label[col_label=="cDC.mig."] <- "cDC mig."

In [ ]:
pdf("result/figures/figure_6/hm_gsea_ctl_d1_d3_wt_vs_ko_gobp.pdf", width=10, height=50)

gsea_hm(mat_1, mat_2, col_label=col_label, col_split=col_split, row_split=row_split, pathway_suffix="WP", fontsize_select=2)

dev.off()

## Heat map log2FC

In [ ]:
# Data base for annotation 
tf <- read.table("/research/peer/fdeckert/reference/animaltfdb/Mus_musculus_TF.txt")[[2]]

lr <- CellChat::CellChatDB.mouse[[1]] %>% dplyr::select(pathway_name, ligand, receptor.symbol, receptor.family, annotation) %>% dplyr::filter(annotation %in% c("Cell-Cell Contact", "Secreted Signaling", "ECM-Receptor")) %>% 
    separate_rows(receptor.symbol, sep=", ") %>% dplyr::distinct() %>%
    separate_rows(ligand, sep=", ") %>% dplyr::distinct() %>% 
    dplyr::rename(pathway=pathway_name, receptor=receptor.symbol, family=receptor.family)

In [ ]:
celltype_select <- c("cMo", "intMo", "RPM", "cDC2", "Basophil")

In [ ]:
# Get sig. DEA results 
mat_1 <- lapply(dea_res_1, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2_thr)})
mat_2 <- lapply(dea_res_2, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2_thr)})
mat_3 <- lapply(dea_res_3, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2_thr)})
mat_4 <- lapply(dea_res_4, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2_thr)})
mat_5 <- lapply(dea_res_5, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2_thr)})
mat_6 <- lapply(dea_res_6, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2_thr)})
mat_7 <- lapply(dea_res_7, function(x) {x %>% dplyr::filter(p_val_adj<=adj_pval_thr & abs(avg_log2FC)>=log2_thr)})

## Shared response genes 

In [ ]:
# DE response genes D1 and D3
de_response_1 <- Map(function(...) do.call(rbind, list(...)), mat_1, mat_2)
de_response_pos_1 <- lapply(de_response_1, function(x) {x %>% group_by(gene) %>% filter(n()>=2 & all(sign(avg_log2FC>0))) %>% dplyr::pull(gene) %>% unique()})
de_response_neg_1 <- lapply(de_response_1, function(x) {x %>% group_by(gene) %>% filter(n()>=2 & all(sign(avg_log2FC<0))) %>% dplyr::pull(gene) %>% unique()})
                     
de_response_2 <- Map(function(...) do.call(rbind, list(...)), mat_3, mat_4)
de_response_pos_2 <- lapply(de_response_2, function(x) {x %>% group_by(gene) %>% filter(n()>=2 & all(sign(avg_log2FC>0))) %>% dplyr::pull(gene) %>% unique()})
de_response_neg_2 <- lapply(de_response_2, function(x) {x %>% group_by(gene) %>% filter(n()>=2 & all(sign(avg_log2FC<0))) %>% dplyr::pull(gene) %>% unique()})

In [ ]:
# Get DE genes between genotypes 
de_genotype <- Map(function(...) do.call(rbind, list(...)), mat_5, mat_6, mat_7)
de_genotype <- lapply(de_genotype, function(x) {x$gene %>% unique()})

In [ ]:
col_samples <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_LysM_cre_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_LysM_cre_CpG_D1", "IFNAR_fl_CpG_D3", "IFNAR_fl_LysM_cre_CpG_D3")
col_split <- c("Ctl", "Ctl", "D1", "D1", "D3", "D3")
col_label <- c("+/+", "cre/+", "+/+", "cre/+", "+/+", "cre/+")

In [ ]:
hm_dea_fetch <- function(celltype_select) {

    genes_pos <- c(de_response_pos_1[[celltype_select]], de_response_pos_2[[celltype_select]]) %>% unique()
    genes_neg <- c(de_response_neg_2[[celltype_select]], de_response_neg_2[[celltype_select]]) %>% unique()

    genes_pos <- genes_pos[!genes_pos %in% de_genotype[[celltype_select]]]
    genes_neg <- genes_neg[!genes_neg %in% de_genotype[[celltype_select]]]

    genes <- c(genes_pos, genes_neg)
    row_split <- c(rep("up", length(genes_pos)), rep("down", length(genes_neg)))

    # Save gene list with figures 
    write.csv(rbind(data.frame(genes=genes_pos, response="Up"), data.frame(genes=genes_pos, response="Down")), paste0("result/dea/scRNAseq/wilcox/fetch/hm_dea_d1_d3_vs_ctl_shared_", tolower(celltype_select), ".csv"), row.names=FALSE)

    # Get pseudobulk counts 
    so_i <- so[, so$celltype_low==celltype_select & so$sample_group %in% col_samples]

    # Get pseudobulk counts 
    mat <- GetAssayData(so_i, assay="RNA", layer="counts")
    mat <- split(t(mat) %>% as.data.frame(), f=so_i$sample_group)

    # Normalize and scale 
    mat <- lapply(mat, colSums) %>% do.call(rbind, .) %>% t()
    mat <- t(t(mat) / colSums(mat)) *1e4
    mat <- log1p(mat)
    mat <- t(scale(t(as.matrix(mat)), center=TRUE, scale=TRUE))

    # Select genes and sample order 
    mat <- mat[genes, col_samples]

    row_genes <- c(
    
        "Mki67", "Pcna", 
        "Spic", 
        "Ly6c2", 
        "Adgre", 
        "Cd163", 
        "Arg1", 
        "Gdf15", 
        "Spi1", 
        "Mertk", "Axl", "Timd4", "Marco",  
        "Cebpa", "Cebpb", "Nr4a1", 
        genes_pos[genes %in% tf], 
        genes_pos[genes %in% lr$ligand], 
        genes_pos[genes %in% lr$receptor], 
        grep("Ifit", genes, value=TRUE), 
        grep("Irf", genes, value=TRUE),
        grep("Oas", genes, value=TRUE), 
        grep("Isg", genes, value=TRUE)
    
    )

    dea_res_hm(mat, row_split=row_split, col_label=col_label, col_split=col_split, celltype=celltype_select, row_genes=row_genes, fontsize_select=2)
    
}

In [ ]:
pdf("result/figures/figure_6/hm_dea_d1_d3_vs_ctl_shared_cmo.pdf", width=2, height=15)

hm_dea_fetch("cMo")

dev.off()

In [ ]:
pdf("result/figures/figure_6/hm_dea_d1_d3_vs_ctl_shared_intmo.pdf", width=2, height=10)

hm_dea_fetch("intMo")

dev.off()

In [ ]:
pdf("result/figures/figure_6/hm_dea_d1_d3_vs_ctl_shared_rpm.pdf", width=2, height=10)

hm_dea_fetch("RPM")

dev.off()

## DE pos response genes 

In [ ]:
# DE response genes D1 and D3
de_response_1 <- Map(function(...) do.call(rbind, list(...)), dea_res_1, dea_res_2)
de_response_1 <- lapply(de_response_1, function(x) {x %>% dplyr::group_by(gene) %>% dplyr::filter(all(sign(avg_log2FC)>0)) %>% dplyr::pull(gene) %>% unique()})

de_response_2 <- Map(function(...) do.call(rbind, list(...)), dea_res_3, dea_res_4)
de_response_2 <- lapply(de_response_2, function(x) {x %>% dplyr::group_by(gene) %>% dplyr::filter(all(sign(avg_log2FC)>0)) %>% dplyr::pull(gene) %>% unique()})

In [ ]:
# Get DE baseline genes 
de_genotype_ctl <- lapply(dea_res_5, function(x) {x %>% dplyr::pull(gene)})

In [ ]:
# Get DE genes between genotypes 
de_genotype <- Map(function(...) do.call(rbind, list(...)), dea_res_6, dea_res_7)
de_genotype_pos <- lapply(de_genotype, function(x) {x %>% group_by(gene) %>% filter(all(sign(avg_log2FC>0))) %>% dplyr::pull(gene) %>% unique()})
de_genotype_neg <- lapply(de_genotype, function(x) {x %>% group_by(gene) %>% filter(all(sign(avg_log2FC<0))) %>% dplyr::pull(gene) %>% unique()})

In [ ]:
col_samples <- c("IFNAR_fl_Baseline_D0", "IFNAR_fl_LysM_cre_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_LysM_cre_CpG_D1", "IFNAR_fl_CpG_D3", "IFNAR_fl_LysM_cre_CpG_D3")
col_split <- c("Ctl", "Ctl", "D1", "D1", "D3", "D3")
col_label <- c("+/+", "cre/+", "+/+", "cre/+", "+/+", "cre/+")

In [ ]:
hm_dea_fetch <- function(celltype_select) {

    genes_pos <- de_genotype_pos[[celltype_select]]
    genes_neg <- de_genotype_neg[[celltype_select]]

    genes_pos <- genes_pos[!genes_pos %in% de_genotype_ctl[[celltype_select]]]
    genes_neg <- genes_neg[!genes_neg %in% de_genotype_ctl[[celltype_select]]]

    genes_pos <- genes_pos[genes_pos %in% c(de_response_1[[celltype_select]])]
    genes_neg <- genes_neg[genes_neg %in% c(de_response_1[[celltype_select]])]

    print(intersect(genes_pos, genes_neg))

    genes <- c(genes_pos, genes_neg)
    row_split <- c(rep("up", length(genes_pos)), rep("down", length(genes_neg)))

    # Save gene list with figures 
    write.csv(rbind(data.frame(genes=genes_pos, response="Up (WT)"), data.frame(genes=genes_pos, response="Down (WT)")), paste0("result/dea/scRNAseq/wilcox/fetch/hm_dea_pos_response_wt_vs_ko_", tolower(celltype_select), ".csv"), row.names=FALSE)

    # Get pseudobulk counts 
    so_i <- so[, so$celltype_low==celltype_select & so$sample_group %in% col_samples]

    # Get pseudobulk counts 
    mat <- GetAssayData(so_i, assay="RNA", layer="counts")
    mat <- split(t(mat) %>% as.data.frame(), f=so_i$sample_group)

    # Normalize and scale 
    mat <- lapply(mat, colSums) %>% do.call(rbind, .) %>% t()
    mat <- t(t(mat) / colSums(mat)) *1e4
    mat <- log1p(mat)
    mat <- t(scale(t(as.matrix(mat)), center=TRUE, scale=TRUE))

    # Select genes and sample order 
    mat <- mat[genes, col_samples]

    row_genes <- c(
    
        "Mki67", "Pcna", 
        "Spic", 
        "Ly6c2", 
        "Adgre", 
        "Cd163", 
        "Arg1", 
        "Gdf15", 
        "Spi1", 
        "Mertk", "Axl", "Timd4", "Marco",  
        "Cebpa", "Cebpb", "Nr4a1", 
        genes_pos[genes %in% tf], 
        genes_pos[genes %in% lr$ligand], 
        genes_pos[genes %in% lr$receptor], 
        grep("Ifit", genes, value=TRUE), 
        grep("Irf", genes, value=TRUE),
        grep("Oas", genes, value=TRUE), 
        grep("Isg", genes, value=TRUE)
    
    )

    dea_res_hm(mat, row_split=row_split, col_label=col_label, col_split=col_split, celltype=celltype_select, row_genes=row_genes, fontsize_select=2)
    
}

In [ ]:
pdf("result/figures/figure_6/hm_dea_pos_response_wt_vs_ko_cmo.pdf", width=2, height=10)

hm_dea_fetch("cMo")

dev.off()

In [ ]:
pdf("result/figures/figure_6/hm_dea_pos_response_wt_vs_ko_intmo.pdf", width=2, height=10)

hm_dea_fetch("intMo")

dev.off()

In [ ]:
pdf("result/figures/figure_6/hm_dea_pos_response_wt_vs_ko_rpm.pdf", width=2, height=10)

hm_dea_fetch("RPM")

dev.off()

## Violine gene expression plots

In [ ]:
gene_select <- c("Ly6c2", "Spic", "Cd163", "Gdf15", "Ccl2", "Slfn4", "Bcl2l1", "Hmox1", "Jun")
celltype_select <- c("cMo", "intMo", "RPM")

In [ ]:
mat <- cbind(so@meta.data[, c("sample_group", "celltype_low"), drop=FALSE], t(GetAssayData(so, assay="RNA", slot="data")[gene_select, ])) %>% 
    pivot_longer(cols=-c(sample_group, celltype_low), names_to="gene", values_to="expression")

In [ ]:
map <- data.frame(
    
    sample_group=c("IFNAR_fl_Baseline_D0", "IFNAR_fl_LysM_cre_Baseline_D0", "IFNAR_fl_CpG_D1", "IFNAR_fl_LysM_cre_CpG_D1", "IFNAR_fl_CpG_D3", "IFNAR_fl_LysM_cre_CpG_D3"),
    time_point=c("Ctl", "Ctl", "D1", "D1", "D3", "D3"),
    genotype=c("+/+", "cre/+", "+/+", "cre/+", "+/+", "cre/+")
)

mat <- mat %>% dplyr::left_join(., map, by=join_by(sample_group)) %>% 
    dplyr::mutate(gene=factor(gene, levels=gene_select))

In [ ]:
genotype <- c("#732E8F", "#D390BF", "#FA8775")
names(genotype) <- c("WT", "+/+", "cre/+")

In [ ]:
options(repr.plot.width=5, repr.plot.height=7.5)

exp_p <- ggplot(mat %>% dplyr::filter(celltype_low %in% celltype_select), aes(x=time_point, y=expression, color=sample_group, fill=sample_group)) + 

        geom_jitter(position=position_jitterdodge(jitter.width=0.20, dodge.width=1.0), shape=21, stroke=0.1, size=0.75, show.legend=FALSE, color="black") + 
        # geom_boxplot(position=position_dodge(width=1.0), width=0.20, color="black", alpha=1, outlier.color=NA) + 
        
        xlab("") + ylab("log(CPTT)") + 
        ggtitle("Spic") + 
        facet_grid(gene~celltype_low, scales="free") + 
        scale_color_manual(values=color$sample_group) + 
        scale_fill_manual(values=color$sample_group)

In [ ]:
pdf("result/figures/figure_6/exp_myeloid.pdf", width=3, height=8)

set_height <- 1 * length(gene_select)
set_width <- 3 * length(celltype_select)

exp_p + ggh4x::force_panelsizes(cols=unit(set_width, "mm"), total_height=unit(set_height, "cm"))

dev.off()

# Volcano plots  

In [ ]:
label <- c("Icam1", "Itga4", "Itgb1", "Vcam1", "Gdf15", "Ccl2", "Tnf", "Il6", "Tgfb1", "Stat1", "Stat2", "Spic", "Irf8", "Siglec1", "Slc40a1", "Hmox1", "Blvrb", "Treml4", "Back1")
# label <- NULL

In [ ]:
source("bin/dea_pl.R")

In [ ]:
v_pl_1 <- v_pl(dea_res_5[["cMo"]], title="cMo", log2_thr=log2_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_Baseline_D0"]), color_neg=unlist(color$sample_group["IFNAR_fl_LysM_cre_Baseline_D0"]), label=label, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5) + theme(legend.position="none") + theme_global_set(size_select=4)
v_pl_2 <- v_pl(dea_res_6[["cMo"]], title="cMo", log2_thr=log2_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_Baseline_D0"]), color_neg=unlist(color$sample_group["IFNAR_fl_LysM_cre_Baseline_D0"]), label=label, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5) + theme(legend.position="none") + theme_global_set(size_select=4)
v_pl_3 <- v_pl(dea_res_7[["cMo"]], title="cMo", log2_thr=log2_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_Baseline_D0"]), color_neg=unlist(color$sample_group["IFNAR_fl_LysM_cre_Baseline_D0"]), label=label, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5) + theme(legend.position="none") + theme_global_set(size_select=4)

In [ ]:
v_pl_4 <- v_pl(dea_res_5[["intMo"]], title="intMo", log2_thr=log2_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_CpG_D1"]), color_neg=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D1"]), label=label, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5) + theme(legend.position="none") + theme_global_set(size_select=4)
v_pl_5 <- v_pl(dea_res_6[["intMo"]], title="intMo", log2_thr=log2_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_CpG_D1"]), color_neg=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D1"]), label=label, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5) + theme(legend.position="none") + theme_global_set(size_select=4)
v_pl_6 <- v_pl(dea_res_7[["intMo"]], title="intMo", log2_thr=log2_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_CpG_D1"]), color_neg=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D1"]), label=label, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5) + theme(legend.position="none") + theme_global_set(size_select=4)

In [ ]:
v_pl_7 <- v_pl(dea_res_5[["RPM"]], title="RPM", log2_thr=log2_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_CpG_D3"]), color_neg=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D3"]), label=label, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5) + theme(legend.position="none") + theme_global_set(size_select=4)
v_pl_8 <- v_pl(dea_res_6[["RPM"]], title="RPM", log2_thr=log2_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_CpG_D3"]), color_neg=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D3"]), label=label, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5) + theme(legend.position="none") + theme_global_set(size_select=4)
v_pl_9 <- v_pl(dea_res_7[["RPM"]], title="RPM", log2_thr=log2_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_CpG_D3"]), color_neg=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D3"]), label=label, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5) + theme(legend.position="none") + theme_global_set(size_select=4)

In [ ]:
v_pl_10 <- v_pl(dea_res_5[["MEP"]], title="MEP", log2_thr=log2_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_CpG_D3"]), color_neg=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D3"]), label=label, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5) + theme(legend.position="none") + theme_global_set(size_select=4)
v_pl_11 <- v_pl(dea_res_6[["MEP"]], title="MEP", log2_thr=log2_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_CpG_D3"]), color_neg=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D3"]), label=label, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5) + theme(legend.position="none") + theme_global_set(size_select=4)
v_pl_12 <- v_pl(dea_res_7[["MEP"]], title="MEP", log2_thr=log2_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_CpG_D3"]), color_neg=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D3"]), label=label, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5) + theme(legend.position="none") + theme_global_set(size_select=4)

In [ ]:
v_pl_13 <- v_pl(dea_res_5[["Proerythroblast"]], title="ProEB", log2_thr=log2_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_CpG_D3"]), color_neg=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D3"]), label=label, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5) + theme(legend.position="none") + theme_global_set(size_select=4)
v_pl_14 <- v_pl(dea_res_6[["Proerythroblast"]], title="ProEB", log2_thr=log2_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_CpG_D3"]), color_neg=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D3"]), label=label, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5) + theme(legend.position="none") + theme_global_set(size_select=4)
v_pl_15 <- v_pl(dea_res_7[["Proerythroblast"]], title="ProEB", log2_thr=log2_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_CpG_D3"]), color_neg=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D3"]), label=label, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5) + theme(legend.position="none") + theme_global_set(size_select=4)

In [ ]:
v_pl_16 <- v_pl(dea_res_5[["Erythroblast"]], title="EB", log2_thr=log2_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_CpG_D3"]), color_neg=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D3"]), label=label, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5) + theme(legend.position="none") + theme_global_set(size_select=4)
v_pl_17 <- v_pl(dea_res_6[["Erythroblast"]], title="EB", log2_thr=log2_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_CpG_D3"]), color_neg=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D3"]), label=label, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5) + theme(legend.position="none") + theme_global_set(size_select=4)
v_pl_18 <- v_pl(dea_res_7[["Erythroblast"]], title="EB", log2_thr=log2_thr, adj_pval_thr=adj_pval_thr, point_size=0.5, color_pos=unlist(color$sample_group["IFNAR_fl_CpG_D3"]), color_neg=unlist(color$sample_group["IFNAR_fl_LysM_cre_CpG_D3"]), label=label, label_size=2, y_limit=4, aspect_ratio=0.8, top_label=5) + theme(legend.position="none") + theme_global_set(size_select=4)

In [ ]:
pdf("result/figures/figure_6/v_pl_cg_vs_ko.pdf", width=3*2, height=6*2)

gridExtra::grid.arrange(
    
    v_pl_1 %>% egg::set_panel_size(., width=unit(3, "cm"), height=unit(2.5, "cm")), 
    v_pl_2 %>% egg::set_panel_size(., width=unit(3, "cm"), height=unit(2.5, "cm")), 
    v_pl_3 %>% egg::set_panel_size(., width=unit(3, "cm"), height=unit(2.5, "cm")), 
    v_pl_4 %>% egg::set_panel_size(., width=unit(3, "cm"), height=unit(2.5, "cm")), 
    v_pl_5 %>% egg::set_panel_size(., width=unit(3, "cm"), height=unit(2.5, "cm")), 
    v_pl_6 %>% egg::set_panel_size(., width=unit(3, "cm"), height=unit(2.5, "cm")), 
    v_pl_7 %>% egg::set_panel_size(., width=unit(3, "cm"), height=unit(2.5, "cm")), 
    v_pl_8 %>% egg::set_panel_size(., width=unit(3, "cm"), height=unit(2.5, "cm")), 
    v_pl_9 %>% egg::set_panel_size(., width=unit(3, "cm"), height=unit(2.5, "cm")), 
    v_pl_10 %>% egg::set_panel_size(., width=unit(3, "cm"), height=unit(2.5, "cm")), 
    v_pl_11 %>% egg::set_panel_size(., width=unit(3, "cm"), height=unit(2.5, "cm")), 
    v_pl_12 %>% egg::set_panel_size(., width=unit(3, "cm"), height=unit(2.5, "cm")), 
    v_pl_13 %>% egg::set_panel_size(., width=unit(3, "cm"), height=unit(2.5, "cm")), 
    v_pl_14 %>% egg::set_panel_size(., width=unit(3, "cm"), height=unit(2.5, "cm")), 
    v_pl_15 %>% egg::set_panel_size(., width=unit(3, "cm"), height=unit(2.5, "cm")), 
    v_pl_16 %>% egg::set_panel_size(., width=unit(3, "cm"), height=unit(2.5, "cm")), 
    v_pl_17 %>% egg::set_panel_size(., width=unit(3, "cm"), height=unit(2.5, "cm")), 
    v_pl_18 %>% egg::set_panel_size(., width=unit(3, "cm"), height=unit(2.5, "cm")), ncol=3

)

dev.off()

# CCI DEA 

In [ ]:
# Load source 
source("bin/cci_pp.R")
source("bin/cci_pl.R")

# Parameter 
pval_prob_thr <- 0.05
p_val_adj_dea <- 0.05
pathway_name_thr <- c("ICAM", "VCAM", "SIRP", "KIT", "TGFb", "TNF", "CXCL", "CCL")

# CellChat results
lr_res <- read.csv("result/cci/lr_res.csv", row.names=1)

# Add reversed Cell-cell interactions 
db <- CellChat::CellChatDB.mouse

# Impute custom interactions
options(stringsAsFactors=FALSE)
interaction <- read.csv("data/reference/cellchat/interaction.csv", row.names=1, na.strings=c(""))
interaction[is.na(interaction)] <- ""
db$interaction <- rbind(db$interaction, interaction)

# Get cell-cell contact interaction 
db <- CellChat::subsetDB(db, search=c("Cell-Cell Contact"), key="annotation")

# Subset lr results for cell-cell contact forward 
lr_res_ccc_fwd <- lr_res %>% dplyr::filter(lr_res$interaction_name %in% db$interaction$interaction_name)

# Reverse cell-cell contact 
lr_res_ccc_rev <- data.frame(

    source=lr_res_ccc_fwd$target, 
    target=lr_res_ccc_fwd$source, 
    interaction_name=paste0(lr_res_ccc_fwd$receptor_symbol, "_", lr_res_ccc_fwd$ligand_symbol), 
    prob=lr_res_ccc_fwd$prob, 
    pval=lr_res_ccc_fwd$pval, 
    pathway_name=lr_res_ccc_fwd$pathway_name, 
    ligand_symbol=lr_res_ccc_fwd$receptor_symbol, 
    receptor_symbol=lr_res_ccc_fwd$ligand_symbol, 
    sample_group=lr_res_ccc_fwd$sample_group
    
)

# Combine results 
lr_res_ccc <- rbind(lr_res_ccc_fwd, lr_res_ccc_rev)

# Assemble LR results 
lr_res <- rbind(lr_res[!lr_res$interaction_name %in% lr_res_ccc$interaction_name, ], lr_res_ccc)

# Subset by interaction pathway
lr_res <- lr_res[lr_res$pathway_name %in% pathway_name_thr, ]
lr_res$pathway_name <- factor(lr_res$pathway_name, levels=pathway_name_thr)

# DEA results 

In [ ]:
dea_mat_5 <- dea_res_5[c("cMo", "intMo", "RPM", "cDC2", "Basophil", "MEP", "Proerythroblast", "Erythroblast")]
dea_mat_5 <- lapply(dea_mat_5, function(x) {list("Ctl"=x)})

In [ ]:
dea_mat_6<- dea_res_6[c("cMo", "intMo", "RPM", "cDC2", "Basophil", "MEP", "Proerythroblast", "Erythroblast")]
dea_mat_6 <- lapply(dea_mat_6, function(x) {list("D1"=x)})

In [ ]:
dea_mat_7 <- dea_res_7[c("cMo", "intMo", "RPM", "cDC2", "Basophil", "MEP", "Proerythroblast", "Erythroblast")]
dea_mat_7 <- lapply(dea_mat_7, function(x) {list("D3"=x)})

In [ ]:
dea_mat <- Map(c, dea_mat_5, dea_mat_6, dea_mat_7)

## PTPG B (+/+) vs D1 (+/+) and D3 (+/+)

## cMo

In [ ]:
source <- c("cMo")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/figure_6/cci_cmo_mep_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("cMo")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/figure_6/cci_cmo_proeb_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("cMo")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/figure_6/cci_cmo_eb_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

## intMo

In [ ]:
source <- c("intMo")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/figure_6/cci_intmo_mep_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("intMo")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/figure_6/cci_intmo_proeb_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("intMo")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/figure_6/cci_intmo_eb_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

## RPM

In [ ]:
source <- c("RPM")
target <- c("MEP")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/figure_6/cci_rpm_mep_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("RPM")
target <- c("Proerythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/figure_6/cci_rpm_proeb_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

In [ ]:
source <- c("RPM")
target <- c("Erythroblast")
sample_group <- c("Ctr"="IFNAR_fl_Baseline_D0", "D1"="IFNAR_fl_CpG_D1", "D3"="IFNAR_fl_CpG_D3")

In [ ]:
lr_res_i <- lr_res_parse(lr_res, source, target, sample_group, pval_prob_thr)
lr_res_i <- lr_res_stretch(lr_res_i)

In [ ]:
pdf("result/figures/figure_6/cci_rpm_eb_ko_vs_cg.pdf", width=5, height=5)

dea_lr_hm(lr_res_i, dea_mat, source, target, fontsize_select=2, lr_de_check=TRUE)

dev.off()

# Interaction plots

In [ ]:
sources <- c("RPM", "intMo", "cMo")
targets <- c("MEP", "Proerythroblast", "Erythroblast")

In [ ]:
cco_1 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_baseline_d0.rds")
cco_2 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_cpg_d1.rds")
cco_3 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_cpg_d3.rds")
cco_4 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_baseline_d0.rds")
cco_5 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_cpg_d1.rds")
cco_6 <- readRDS("data/scRNAseq/cci/cco_ifnar_fl_lysm_cre_cpg_d3.rds")

In [ ]:
cco_1 <- subsetCellChat(object=cco_1, idents.use=c(sources, targets))
cco_2 <- subsetCellChat(object=cco_2, idents.use=c(sources, targets))
cco_3 <- subsetCellChat(object=cco_3, idents.use=c(sources, targets))
cco_4 <- subsetCellChat(object=cco_4, idents.use=c(sources, targets))
cco_5 <- subsetCellChat(object=cco_5, idents.use=c(sources, targets))
cco_6 <- subsetCellChat(object=cco_6, idents.use=c(sources, targets))

In [ ]:
levels(cco_1@idents) <- c(targets, sources)
levels(cco_2@idents) <- c(targets, sources)
levels(cco_3@idents) <- c(targets, sources)
levels(cco_4@idents) <- c(targets, sources)
levels(cco_5@idents) <- c(targets, sources)
levels(cco_6@idents) <- c(targets, sources)

In [ ]:
color_use <- color$celltype_low[names(color$celltype_low) %in% c(sources, targets)]

In [ ]:
cco_list <- list(cco_1, cco_2, cco_3, cco_4, cco_5, cco_6)

q_prob_thr <- quantile(c(cco_1@net$prob, cco_2@net$prob, cco_3@net$prob), 0.90)
for (i in seq_along(cco_list)) {
    
    prob_arr <- cco_list[[i]]@net$prob
    prob_arr[prob_arr > q_prob_thr] <- q_prob_thr
    cco_list[[i]]@net$prob <- prob_arr
    
}

In [ ]:
pdf("result/figures/figure_6/cp_cci_net_ctl_ko.pdf", width=2.0, height=2.0)

    replayPlot(netVisual_chord_gene(cco_list[[4]], slot.name="net", sources.use=sources, targets.use=targets, signaling=pathway_name_thr, color.use=color_use, title.name=NULL, show.legend=FALSE, small.gap=1, big.gap=5, lab.cex=0.5))

dev.off()

In [ ]:
pdf("result/figures/figure_6/cp_cci_net_d1_ko.pdf", width=2.0, height=2.0)

    replayPlot(netVisual_chord_gene(cco_list[[5]], slot.name="net", sources.use=sources, targets.use=targets, signaling=pathway_name_thr, color.use=color_use, title.name=NULL, show.legend=FALSE, small.gap=1, big.gap=5, lab.cex=0.5))

dev.off()

In [ ]:
pdf("result/figures/figure_6/cp_cci_net_d3_ko.pdf", width=2.0, height=2.0)

    replayPlot(netVisual_chord_gene(cco_list[[6]], slot.name="net", sources.use=sources, targets.use=targets, signaling=pathway_name_thr, color.use=color_use, title.name=NULL, show.legend=FALSE, small.gap=1, big.gap=5, lab.cex=0.5))

dev.off()